In [1]:
import pandas as pd
import numpy as np
import os
import locale
import statsmodels.api as sm

# **1. Daten laden**
# 
# Überprüfen, ob alle benötigten Dateien vorhanden sind
required_files = [
    "umsatzdaten_gekuerzt.csv",
    "kiwo.csv",
    "wetter.csv",
    "Data Import and Preparation/feiertage.csv"
]
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Datei '{file}' nicht gefunden.")

# Daten laden
umsatzdaten = pd.read_csv("umsatzdaten_gekuerzt.csv")
kiwo_data = pd.read_csv("kiwo.csv")
wetter_data = pd.read_csv("wetter.csv")
feiertage = pd.read_csv("Data Import and Preparation/feiertage.csv")  # Enthält Spalten 'Datum' und 'Feiertag'

In [2]:
# **2. Daten vorverarbeiten**

# Konvertiere 'Datum' in datetime für alle Datensätze
umsatzdaten['Datum'] = pd.to_datetime(umsatzdaten['Datum'], errors='coerce') 
kiwo_data['Datum'] = pd.to_datetime(kiwo_data['Datum'], errors='coerce')
wetter_data['Datum'] = pd.to_datetime(wetter_data['Datum'], errors='coerce')

# Locale auf Deutsch setzen
try:
    locale.setlocale(locale.LC_TIME, 'de_DE.UTF-8')
except locale.Error:
    print("Locale-Einstellung auf 'de_DE.UTF-8' fehlgeschlagen. Weiter ohne Locale.")

# Feiertage-Daten anpassen (Jahr zufällig zwischen 2013 und 2018 setzen)
def adjust_year(date_str):
    if isinstance(date_str, str) and date_str.strip():
        month_day = date_str.replace('.', '').strip()
        year = np.random.choice([2013, 2014, 2015, 2016, 2017, 2018])
        return f"{month_day} {year}"
    else:
        return np.nan

feiertage['Datum'] = feiertage['Datum'].apply(adjust_year)
feiertage['Datum'] = pd.to_datetime(feiertage['Datum'], format='%d %b %Y', errors='coerce')
feiertage = feiertage.dropna(subset=['Datum'])

In [3]:
# **3. Daten zusammenführen**

merged_data = umsatzdaten.merge(kiwo_data, on='Datum', how='left')
merged_data = merged_data.merge(wetter_data, on='Datum', how='left')
merged_data = merged_data.merge(feiertage[['Datum', 'Name']], on='Datum', how='left')

In [4]:
# **4. Zusätzliche Variablen erstellen**

# Wochentag hinzufügen
merged_data['Wochentag'] = merged_data['Datum'].dt.day_name()

# Monat als numerischen Wert hinzufügen
merged_data['Monat'] = merged_data['Datum'].dt.month

# Binäre Feiertags-Flagge hinzufügen
merged_data['Ist_Feiertag'] = merged_data['Name'].notnull().astype(int)

# Quartal hinzufügen
merged_data['Quartal'] = merged_data['Datum'].dt.quarter

# Saisonale Variable hinzufügen
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Frühling"
    elif month in [6, 7, 8]:
        return "Sommer"
    else:
        return "Herbst"

merged_data['Saison'] = merged_data['Monat'].apply(get_season)

# Wochenend-Flagge hinzufügen
merged_data['Ist_Wochenende'] = merged_data['Wochentag'].isin(['Saturday', 'Sunday']).astype(int)

# Feiertagstypen kategorisieren
merged_data['Feiertag_Christlich'] = merged_data['Name'].str.contains('Christlich', na=False).astype(int)
merged_data['Feiertag_National'] = merged_data['Name'].str.contains('National', na=False).astype(int)

# Temperatur kategorisieren
def categorize_temp(temp):
    if temp <= 10:
        return 'Kalt'
    elif temp <= 25:
        return 'Mild'
    return 'Heiß'

merged_data['Temp_Kategorie'] = merged_data['Temperatur'].apply(categorize_temp)

# Resttage im Monat
merged_data['Resttage_Monat'] = merged_data['Datum'].dt.days_in_month - merged_data['Datum'].dt.day

# Dummy für Arbeitstage
merged_data['Ist_Arbeitstag'] = (~merged_data['Ist_Wochenende'] & ~merged_data['Ist_Feiertag']).astype(int)

# Fehlende Werte behandeln
merged_data.fillna(0, inplace=True)

In [5]:
# **5. Datensatz aufteilen**
train_data = merged_data[(merged_data['Datum'] >= '2013-07-01') & (merged_data['Datum'] <= '2017-07-31')]
valid_data = merged_data[(merged_data['Datum'] >= '2017-08-01') & (merged_data['Datum'] <= '2018-07-31')]

# Exportiere die Datensätze
output_dir = "data"
os.makedirs(output_dir, exist_ok=True)
train_data.to_csv(f"{output_dir}/train.csv", index=False)
valid_data.to_csv(f"{output_dir}/validation.csv", index=False)

In [6]:
# **6. Lineare Regression durchführen**
features = ['Temperatur', 'Feiertag_Christlich', 'Feiertag_National', 'Ist_Wochenende', 'Quartal']
X_train = train_data[features]
y_train = train_data['Umsatz']

# Konstante hinzufügen
X_train = sm.add_constant(X_train)

# Lineare Regression
model = sm.OLS(y_train, X_train).fit()

# Modellzusammenfassung
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     145.9
Date:                 Do, 05 Dez 2024   Prob (F-statistic):          2.44e-120
Time:                        15:59:08   Log-Likelihood:                -47783.
No. Observations:                7493   AIC:                         9.558e+04
Df Residuals:                    7488   BIC:                         9.561e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                 145.8471    

In [13]:
# **7. Exportierte Dateien und Ergebnisse dokumentieren**
readme_content = """
# Baseline Model: Umsatzvorhersage

## 1. Datenaufbereitung
- Zusätzliche Variablen: Feiertagskategorien, Wetterkategorien, Tage seit letztem Feiertag etc.
- Datenaufteilung in Trainings- und Validierungsdatensätze.

## 2. Lineares Modell
- Zielvariable: `Umsatz`
- Features: Temperatur, Feiertage, Quartal, Wochenend-Flagge etc.
- Adjustiertes R^2: <Ergebnis hier einfügen>

## 3. Ergebnisse
- Modellzusammenfassung:
{model.summary()}
"""

with open(f"{output_dir}/README.md", "w", encoding="utf-8") as f:
    f.write(readme_content)